<a href="https://colab.research.google.com/github/ad-batrisyia/money-laundering-detection-ml/blob/main/notebooks/03_Random_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from PIL import Image
from scipy.stats import skew
from matplotlib.transforms import Bbox
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path where you saved it
load_path = '/content/drive/MyDrive/Money Laundering Detection/PreprocessedData.csv'

# Load the CSV
df = pd.read_csv(load_path)

df.head()

,Amount,Payment_type,Is_laundering,TimeOfDay,Is_Weekend,Tx_per_Day,Tx_Interval,Avg_Amount_Sent,Pair_bank_location,Laundering_Structure
0,1459.15,1,0,0,0,1,0.0,1857.866667,304,2
1,6019.64,5,0,0,0,102,0.0,5889.320323,303,6
2,14328.44,3,0,0,0,3,0.0,10167.677647,304,6
3,11895.00,0,0,0,0,9,0.0,11937.773333,304,5
4,115.25,1,0,0,0,1,0.0,1068.905000,304,2


In [ ]:
df.shape

(9504852, 10)

In [ ]:
X = df.drop(columns=['Is_laundering'], axis=1)
y = df['Is_laundering']

## Split Train-Validation-Test 70:15:15 (Random Sampling)

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# ====================
# 2) Train-Validate-Test Split
# ====================
# First split off test set (15%)
x_trainval, x_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# Then split trainval into train (70%) + validation (15%)
x_train, x_val, y_train, y_val = train_test_split(
    x_trainval, y_trainval, test_size=0.176, random_state=42
)
# (0.176 of 0.85 = 0.15 total)

print("x_train:", x_train.shape, "y_train:", y_train.shape)
print("x_val:", x_val.shape, "y_val:", y_val.shape)
print("x_test:", x_test.shape, "y_test:", y_test.shape)

x_train: (6657198, 9) y_train: (6657198,)
x_val: (1421926, 9) y_val: (1421926,)
x_test: (1425728, 9) y_test: (1425728,)


In [ ]:
# Class distribution in training set
print("\nClass distribution in y_train:")
print(y_train.value_counts())

# Class distribution in validation set
print("\nClass distribution in y_train:")
print(y_val.value_counts())

# Class distribution in testing set
print("\nClass distribution in y_test:")
print(y_test.value_counts())


Class distribution in y_train:
Is_laundering
0    6650237
1       6961
Name: count, dtype: int64

Class distribution in y_train:
Is_laundering
0    1420453
1       1473
Name: count, dtype: int64

Class distribution in y_test:
Is_laundering
0    1424289
1       1439
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix)

def evaluate_model(name, y_true, y_pred):
    print(f"\n=== {name} ===")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

# === Random Forest Baseline ===
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(x_train, y_train)
y_val_pred_rf = rf.predict(x_val)

# === XGBoost Baseline ===
xgb = XGBClassifier(
    random_state=42,
    eval_metric="logloss",
    n_jobs=-1)

xgb.fit(x_train, y_train)
y_val_pred_xgb = xgb.predict(x_val)

In [ ]:
evaluate_model("Random Forest", y_val, y_val_pred_rf)
evaluate_model("XGBoost", y_val, y_val_pred_xgb)


=== Random Forest ===
Accuracy: 0.9997
Classification Report:
              precision    recall  f1-score   support

           0     0.9997    1.0000    0.9998   1420453
           1     0.9829    0.7033    0.8199      1473

    accuracy                         0.9997   1421926
   macro avg     0.9913    0.8517    0.9099   1421926
weighted avg     0.9997    0.9997    0.9997   1421926

Confusion Matrix:
[[1420435      18]
 [    437    1036]]

=== XGBoost ===
Accuracy: 0.9995
Classification Report:
              precision    recall  f1-score   support

           0     0.9996    0.9999    0.9997   1420453
           1     0.8412    0.6327    0.7222      1473

    accuracy                         0.9995   1421926
   macro avg     0.9204    0.8163    0.8610   1421926
weighted avg     0.9995    0.9995    0.9995   1421926

Confusion Matrix:
[[1420277     176]
 [    541     932]]
